In [ ]:
import xarray as xa
import numpy as np
import pandas as pd
import geopandas
import spatialpandas
import shapely
from shapely.geometry import LineString

import datashader as ds
from datashader.transfer_functions import shade, stack, dynspread, set_background

from xrspatial import a_star_search

## Load data

The road network used in this example notebook can be downloaded from:

https://data.austintexas.gov/Locations-and-Maps/Street-Centerline/m5w3-uea6

In [ ]:
streets = geopandas.read_file('./data/Street_Centerline.zip')
streets = streets.to_crs('EPSG:4326')
streets = streets.explode('geometry').reset_index(drop=True)
streets_spd = spatialpandas.GeoDataFrame(streets, geometry='geometry')

## Define study area (find range of x and y)

In [ ]:
xmin, ymin, xmax, ymax = (streets.geometry.bounds.minx.min(),
                          streets.geometry.bounds.miny.min(),
                          streets.geometry.bounds.maxx.max(),
                          streets.geometry.bounds.maxy.max())
xrange = (xmin, xmax)
yrange = (ymin, ymax)
xrange, yrange

In [ ]:
H, W = 600, 800
cvs = ds.Canvas(plot_width=W, plot_height=H,
                x_range=xrange, y_range=yrange)

street_agg = cvs.line(streets_spd, geometry='geometry')
street_shaded = dynspread(shade(street_agg, cmap=['salmon']))

# Pick two locations
start = (-97.73662282, 30.08214069)
goal = (-97.63753489650679, 30.1765660638542)

location_df = pd.DataFrame({'x': [start[0], goal[0]], 'y': [start[1], goal[1]]})
location_agg = cvs.points(location_df, 'x', 'y')
location_shaded = dynspread(shade(location_agg, cmap=['lime']), threshold=1, max_px=5)

set_background(stack(street_shaded, location_shaded), 'black')

## Shortest path using A* from start location to goal location

In [ ]:
# find the path from start to goal, 
# barriers are uncrossable cells. In this case, they are cells with a value of 0

path_agg = a_star_search(street_agg, start, goal, barriers=[0], snap_start=True, snap_goal=True)

path_shaded = dynspread(shade(path_agg, cmap=['green']))
set_background(stack(street_shaded, path_shaded, location_shaded), 'black')